In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import aniensemblestats as aes
import pyaniasetools as aat

import pyanitools as pyt

import numpy as np
import ase

#from ase.neb import NEBtools
from ase.io import read, write

import matplotlib
import matplotlib as mpl
from matplotlib import gridspec 
import matplotlib.pyplot as plt

import hdnntools as hdt

from scipy import stats

import os

van Der Waals correction will be unavailable. Please install ased3


/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#ntdir = '/home/jsmith48/scratch/TZData/train_L2T1E-6_ba/'
#ntdir = '/home/jsmith48/scratch/TZData_force/train_msecost_1.0_0.25/'
ntdir1 = '/home/jsmith48/scratch/ccsd_extrapolation/learning_cases/dft_DZ_retrain/retrain_dft_f02/'
cns1 = ntdir1 + 'rHCNO-4.6R_16-3.1A_a4-8.params'
sae1 = ntdir1 + 'sae_linfit.dat'
nnf1 = ntdir1 + 'train'
Nn1 = 5

ntdir2 = '/home/jsmith48/scratch/ccsd_extrapolation/learning_cases/ful_train/'
cns2 = ntdir2 + 'rHCNO-4.6R_16-3.1A_a4-8.params'
sae2 = ntdir2 + 'sae_linfit.dat'
nnf2 = ntdir2 + 'train'
Nn2 = 5

ntdir3 = '/home/jsmith48/scratch/ANI-2x_retrain/model2/'
cns3 = ntdir3 + 'rHCNOSFCl-4.6R_16-3.1A_a4-8.params'
sae3 = ntdir3 + 'sae_linfit.dat'
nnf3 = ntdir3 + 'train'
Nn3 = 5

dftsae = '/home/jsmith48/scratch/ccsd_extrapolation/atomization_files/sae_wb97x-631gd.dat'
ccssae = '/home/jsmith48/scratch/ccsd_extrapolation/atomization_files/ccsd_atmz.dat'

ts = '/home/jsmith48/scratch/ccsd_extrapolation/h5files_holdout_split/testset/r10_ccsd_test.h5'

dftdir = '/home/jsmith48/scratch/ccsd_extrapolation/h5files_holdout_split/testset/xyz/'

In [3]:
dftfiles = [f for f in os.listdir(dftdir) if f[-4:] == '.dat']
dftdata = dict()
for f in dftfiles:
    data = hdt.readncdatall(dftdir+f)
    dftdata.update({f.rsplit('.',1)[0]:data})
    

In [4]:
# Set NC
aens_1 = aat.anicrossvalidationconformer(cns1, sae1, nnf1, Nn1, 2)
aens_2 = aat.anicrossvalidationconformer(cns2, sae2, nnf2, Nn2, 2)
aens_3 = aat.anicrossvalidationconformer(cns3, sae3, nnf3, Nn3, 2)

In [5]:
adl = pyt.anidataloader(ts)

In [6]:
Eal=[]
dEal=[]
Emin=[]

Na = []
for i,data in enumerate(adl):
    #if i  == 100:
    #    break
    if data['path'][1:] in dftdata:
        
        # Get reference data
        X = np.array(data['coordinates'],dtype=np.float32)
        S = data['species']
        E = hdt.hatokcal*data['energies']
        Na.append(len(S))
        
        # Compute SAE lin fit
        Esae= hdt.hatokcal*hdt.compute_sae(file=ccssae,spc=S)
        Esae_dft= hdt.hatokcal*hdt.compute_sae(file=dftsae,spc=S)
        
        # Define Eref
        Eref = E-Esae
        
        # Define DFT comparison
        dft = dftdata[data['path'][1:]]
        Edft = hdt.hatokcal*dft['energies']-Esae_dft

        # Compute energies and forces
        Ea1_t,Fa1_t,Sa1 = aens_1.compute_energyandforce_conformations(X,S)
        Ea2_t,Fa2_t,Sa2 = aens_2.compute_energyandforce_conformations(X,S)
        Ea3_t,Fa3_t,Sa3 = aens_3.compute_energyandforce_conformations(X,S)

        idx = np.where(Sa1 < 500.0)
        
        print('Computing:',i,len(S),data['path'],idx[0].size)
        
        if idx[0].size != 0:

            # Define NN shifted energies
            Eal1 = Ea1_t[idx]-Esae
            Eal2 = Ea2_t[idx]-Esae
            Eal3 = Ea3_t[idx]-Esae_dft

            # Define NN E distmats
            dEal1 = hdt.calculatedmat(Ea1_t[idx])
            dEal2 = hdt.calculatedmat(Ea2_t[idx])
            dEal3 = hdt.calculatedmat(Ea3_t[idx])

            # Store E mins
            Emin.append(np.array([[Eref[idx].min(),Eal1.min(),Eal2.min(),Eal3.min(),Edft[idx].min()]]))

            # Store E and dE
            Eal.append(np.vstack([Eref[idx],Eal1,Eal2,Eal3,Edft[idx]]))
            dEal.append(np.vstack([hdt.calculatedmat(Eref[idx]),dEal1,dEal2,dEal3,hdt.calculatedmat(Edft[idx])]))
            #print(dEal[-1].shape)
        
Eal = np.hstack(Eal)
dEal = np.hstack(dEal)
#print(dEal.shape,dEal)
Na = np.array(Na)
Emin=np.vstack(Emin)

Computing: 0 16 /mol_10076 10
Computing: 1 17 /mol_1008 10
Computing: 2 20 /mol_10080 10
Computing: 3 23 /mol_10102 10
Computing: 4 24 /mol_10110 10
Computing: 5 22 /mol_10111 10
Computing: 6 22 /mol_10117 10
Computing: 7 21 /mol_10124 10
Computing: 8 20 /mol_10132 10
Computing: 9 21 /mol_10179 10
Computing: 10 22 /mol_10190 10
Computing: 11 20 /mol_10198 10
Computing: 12 19 /mol_10200 10
Computing: 13 20 /mol_10223 10
Computing: 14 18 /mol_10233 10


RuntimeError: unidentifiable C++ exception

In [ ]:
#print(Eal1.shape,len(Edl))
print(Emin[:,0].min(),Emin[:,1].min(),Emin[:,2].min(),Emin[:,3].min())

In [ ]:
print('E error')
rms = hdt.calculaterootmeansqrerror(Eal[1,:],Eal[0,:])
mae = hdt.calculatemeanabserror(Eal[1,:],Eal[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(Eal[2,:],Eal[0,:])
mae = hdt.calculatemeanabserror(Eal[2,:],Eal[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(Eal[3,:],Eal[0,:])
mae = hdt.calculatemeanabserror(Eal[3,:],Eal[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(Eal[4,:],Eal[0,:])
mae = hdt.calculatemeanabserror(Eal[4,:],Eal[0,:])
print(mae,rms)

In [ ]:
print('dE error')
rms = hdt.calculaterootmeansqrerror(dEal[1,:],dEal[0,:])
mae = hdt.calculatemeanabserror(dEal[1,:],dEal[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(dEal[2,:],dEal[0,:])
mae = hdt.calculatemeanabserror(dEal[2,:],dEal[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(dEal[3,:],dEal[0,:])
mae = hdt.calculatemeanabserror(dEal[3,:],dEal[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(dEal[4,:],dEal[0,:])
mae = hdt.calculatemeanabserror(dEal[4,:],dEal[0,:])
print(mae,rms)

In [ ]:
#dft_shift = Edl.mean()-Eal3.mean()

aes.plot_corr_dist(dEal[1,:],dEal[0,:],xlabel=r'$E_{CCSD} (kcal \times mol^{-1})$',ylabel=r'$E_{ANI} (kcal \times mol^{-1})$')
aes.plot_corr_dist(dEal[2,:],dEal[0,:],xlabel=r'$E_{CCSD} (kcal \times mol^{-1})$',ylabel=r'$E_{ANI} (kcal \times mol^{-1})$')
aes.plot_corr_dist(dEal[3,:],dEal[0,:],xlabel=r'$E_{CCSD} (kcal \times mol^{-1})$',ylabel=r'$E_{DFT} (kcal \times mol^{-1})$')
aes.plot_corr_dist(dEal[4,:],dEal[0,:],xlabel=r'$E_{CCSD} (kcal \times mol^{-1})$',ylabel=r'$E_{DFT} (kcal \times mol^{-1})$')
aes.plot_corr_dist(dEal[3,:],dEal[4,:],xlabel=r'$E_{CCSD} (kcal \times mol^{-1})$',ylabel=r'$E_{DFT} (kcal \times mol^{-1})$')

In [ ]:
f = plt.subplots(figsize=[20,15])
gs  = gridspec.GridSpec(2, 2, width_ratios=[.85, .85])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
ax3 = plt.subplot(gs[2])
ax4 = plt.subplot(gs[3])

print((Eal[0,:]-Eal[3,:]).mean(),(Eal[0,:]-Eal[4,:]).mean())

bin1 = aes.plot_corr_dist_axes(ax1,Eal[0,:],Eal[1,:],labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T)/CBS Transfer NN',cmap=mpl.cm.viridis)
bin2 = aes.plot_corr_dist_axes(ax2,Eal[0,:],Eal[2,:],labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T)/CBS Trained NN',cmap=mpl.cm.viridis)
bin3 = aes.plot_corr_dist_axes(ax3,Eal[0,:],Eal[3,:]+(Eal[0,:]-Eal[3,:]).mean(),labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{ANI} (kcal \times mol^{-1})$',plabel='DFT/DZ Trained NN',cmap=mpl.cm.viridis)
bin4 = aes.plot_corr_dist_axes(ax4,Eal[0,:],Eal[4,:]+(Eal[0,:]-Eal[4,:]).mean(),labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{DFT} (kcal \times mol^{-1})$',plabel='DFT/DZ QM',cmap=mpl.cm.viridis)

f[0].subplots_adjust(right=0.8)
cbar_ax = f[0].add_axes([0.825, 0.15, 0.025, 0.7])
cb1 = f[0].colorbar(bin1[-1], cmap=mpl.cm.viridis, cax=cbar_ax)

cb1.set_label('Count', fontsize=20)
plt.show()

In [ ]:
#slope, intercept, r_value, p_value, std_err = stats.linregress(hdt.hatokcal*(Edl),hdt.hatokcal*(Eal2))
#print(r_value**2)

In [ ]:
f = plt.subplots(figsize=[23,18])
gs  = gridspec.GridSpec(2, 2, width_ratios=[.85, .85])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
ax3 = plt.subplot(gs[2])
ax4 = plt.subplot(gs[3])

bin1 = aes.plot_corr_dist_axes(ax1,dEal[0,:],dEal[1,:],labelx=r'$\Delta E_{CCSD(T)/CBS}(kcal \times mol^{-1})$',labely=r'$\Delta E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T) Transfer NN',cmap=mpl.cm.viridis)
bin2 = aes.plot_corr_dist_axes(ax2,dEal[0,:],dEal[2,:],labelx=r'$\Delta E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$\Delta E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T) Trained NN',cmap=mpl.cm.viridis)
bin3 = aes.plot_corr_dist_axes(ax3,dEal[0,:],dEal[3,:],labelx=r'$\Delta E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$\Delta E_{ANI} (kcal \times mol^{-1})$',plabel='DFT/DZ Trained NN',cmap=mpl.cm.viridis)
bin4 = aes.plot_corr_dist_axes(ax4,dEal[0,:],dEal[4,:],labelx=r'$\Delta E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$\Delta E_{DFT/DZ} (kcal \times mol^{-1})$',plabel='DFT/DZ',cmap=mpl.cm.viridis)

f[0].subplots_adjust(right=0.8)
cbar_ax = f[0].add_axes([0.82, 0.15, 0.025, 0.7])
cb1 = f[0].colorbar(bin4[-1], cmap=mpl.cm.viridis, cax=cbar_ax)

cb1.set_label('Count', fontsize=20)
plt.show()

In [ ]:
f = plt.subplots(figsize=[32,10])
gs  = gridspec.GridSpec(1, 3, width_ratios=[.85, .85, 1.0])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
ax3 = plt.subplot(gs[2])

Emin_ref = hdt.calculatedmat(Emin[:,0])

bin1 = aes.plot_corr_dist_axes(ax1,hdt.calculatedmat(Emin[:,1]),Emin_ref,labelx=r'$\Delta E_{CCSD(T)/CBS}(kcal \times mol^{-1})$',labely=r'$\Delta E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T) from DFT/DZ',cmap=mpl.cm.viridis)
bin2 = aes.plot_corr_dist_axes(ax2,hdt.calculatedmat(Emin[:,2]),Emin_ref,labelx=r'$\Delta E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$\Delta E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T) Only',cmap=mpl.cm.viridis)
bin3 = aes.plot_corr_dist_axes(ax3,hdt.calculatedmat(Emin[:,3]),Emin_ref,labelx=r'$\Delta E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$\Delta E_{ANI} (kcal \times mol^{-1})$',plabel='DFT/DZ',cmap=mpl.cm.viridis)
cb1 = f[0].colorbar(bin1[-1], cmap=mpl.cm.viridis)
cb1.set_label('Count', fontsize=20)
plt.show()

In [ ]:
plt.hist(Na, bins=16)
plt.show()

In [ ]:
# f = plt.subplots(figsize=[24,10])
# gs  = gridspec.GridSpec(1, 2, width_ratios=[.85, 1.0])
# ax1 = plt.subplot(gs[0])

# ax1.hist(dEdl-dEal1,bins=75, range=[-10,10], edgecolor='black', linewidth=1.2, ls='dashed', fc=(0, 0, 1, 0.4),label='Layer 1 and 3 frozen')
# ax1.hist(dEdl-dEal2,bins=75, range=[-10,10], edgecolor='black', linewidth=1.2, ls='dashed', fc=(1, 0, 0, 0.4),label='Full model retrained')
# ax1.set_title('Error Distributions', fontsize=24)
# ax1.legend(fontsize=16)
# ax1.set_xlabel(r'$\Delta E$ ($kcal \times mol^{-1}$)',fontsize=20)
# ax1.set_ylabel('Count',fontsize=20)

# ax2 = plt.subplot(gs[1])
# ax2.hist(Na,bins=16, ls='dashed', edgecolor='black', linewidth=1.2, fc=(1, 0, 0, 0.4))
# ax2.set_title('System sizes', fontsize=24)
# ax2.set_xlabel('Numer of atoms',fontsize=20)

# plt.show()

In [ ]:
adl = pyt.anidataloader('/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/s66x8/s66x8_wb97x6-31gd.h5')

In [ ]:
sdir = '/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/s66x8/xyz_splits/'

In [ ]:
ani_opt1 = aat.anienscomputetool(cns1, sae1, nnf1, Nn1, 2)
ani_opt2 = aat.anienscomputetool(cns2, sae2, nnf2, Nn2, 2)
ani_opt3 = aat.anienscomputetool(cns3, sae3, nnf3, Nn3, 2)

In [ ]:
ccsd = open('/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/s66x8/ccsd_t_energies.dat')
R = [0.9, 0.95, 1.0, 1.05, 1.1, 1.25, 1.5, 2.0]
Es66 = []
for j,data in enumerate(adl):
    print(data['path'])
    
    if j < 66:

        Ecc = np.zeros(8,dtype=np.float64)
        for i in range(8):
            Ecc[i] = float(ccsd.readline().split()[-1])

        S = data['species']
        X = data['coordinates']
        Edft = hdt.hatokcal*data['energies']

        # Compute SAE lin fit
        Esae_ccs= hdt.hatokcal*hdt.compute_sae(file=ccssae,spc=S)
        Esae_dft= hdt.hatokcal*hdt.compute_sae(file=dftsae,spc=S)
        
        #print(Edft-Edft.min(),Ecc-Ecc.min())

        Ea1_t,Fa1_t,Sa1 = aens_1.compute_energyandforce_conformations(X,S)
        Ea2_t,Fa2_t,Sa2 = aens_2.compute_energyandforce_conformations(X,S)
        Ea3_t,Fa3_t,Sa3 = aens_3.compute_energyandforce_conformations(X,S)

        #print(os.listdir(sdir))
        
        X1,S1,Na1,C1 = hdt.readxyz2(sdir+'s66x8_'+str(j).zfill(2)+'-1.xyz')
        X1o,CNV = ani_opt1.optimize_molecule(X1[0],S1,fmax=0.0001)
        Ea1s1_t,Fa1s1_t,Sas11 = aens_1.compute_energyandforce_conformations(X1o[np.newaxis,:,:],S1)
        X1o,CNV = ani_opt2.optimize_molecule(X1[0],S1,fmax=0.0001)
        Ea2s1_t,Fa2s1_t,Sas12 = aens_2.compute_energyandforce_conformations(X1o[np.newaxis,:,:],S1)
        X1o,CNV = ani_opt3.optimize_molecule(X1[0],S1,fmax=0.0001)
        Ea3s1_t,Fa3s1_t,Sas13 = aens_3.compute_energyandforce_conformations(X1o[np.newaxis,:,:],S1)

        X2,S2,Na2,C2 = hdt.readxyz2(sdir+'s66x8_'+str(j).zfill(2)+'-2.xyz')
        X2o,CNV = ani_opt1.optimize_molecule(X2[0],S2,fmax=0.0001)
        Ea1s2_t,Fa1s2_t,Sas21 = aens_1.compute_energyandforce_conformations(X2o[np.newaxis,:,:],S2)
        X2o,CNV = ani_opt1.optimize_molecule(X2[0],S2,fmax=0.0001)
        Ea2s2_t,Fa2s2_t,Sas22 = aens_2.compute_energyandforce_conformations(X2o[np.newaxis,:,:],S2)
        X2o,CNV = ani_opt1.optimize_molecule(X2[0],S2,fmax=0.0001)
        Ea3s2_t,Fa3s2_t,Sas23 = aens_3.compute_energyandforce_conformations(X2o[np.newaxis,:,:],S2)
        
        print(Ea1s1_t+Ea1s2_t, Ea1_t)
        
        Ea1s = Ea1s1_t + Ea1s2_t
        Ea2s = Ea2s1_t + Ea2s2_t
        Ea3s = Ea3s1_t + Ea3s2_t
        
        #plt.plot(Edft,label='DFT/DZ')
        plt.plot(R,Ecc,label='CCSD(T)')
        plt.plot(R,Ea1_t-Ea1_t[-1]+Ecc[-1],label='ANI TL')
        plt.plot(R,Ea2_t-Ea2_t[-1]+Ecc[-1],label='ANI CC')
        plt.plot(R,Ea3_t-Ea3_t[-1]+Ecc[-1],label='ANI DT')
        
        Es66.append(np.array([Ecc,Ea1_t-Ea1_t[-1]+Ecc[-1],Ea2_t-Ea2_t[-1]+Ecc[-1],Ea3_t-Ea3_t[-1]+Ecc[-1],Edft-Edft.min()]))

        plt.legend()
        plt.show()
Es66 = np.hstack(Es66)

In [ ]:
print('E error -- s66x8 hydrogen bonding error')
print('        MAE               RMSE')
rms = hdt.calculaterootmeansqrerror(Es66[1,:],Es66[0,:])
mae = hdt.calculatemeanabserror(Es66[1,:],Es66[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(Es66[2,:],Es66[0,:])
mae = hdt.calculatemeanabserror(Es66[2,:],Es66[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(Es66[3,:],Es66[0,:])
mae = hdt.calculatemeanabserror(Es66[3,:],Es66[0,:])
print(mae,rms)
rms = hdt.calculaterootmeansqrerror(Es66[4,:],Es66[0,:])
mae = hdt.calculatemeanabserror(Es66[4,:],Es66[0,:])
print(mae,rms)

In [ ]:
f = plt.subplots(figsize=[20,15])
gs  = gridspec.GridSpec(2, 2, width_ratios=[.85, .85])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
ax3 = plt.subplot(gs[2])
ax4 = plt.subplot(gs[3])

bin1 = aes.plot_corr_dist_axes(ax1,Es66[0,:],Es66[1,:],labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T)/CBS Transfer NN',cmap=mpl.cm.viridis)
bin2 = aes.plot_corr_dist_axes(ax2,Es66[0,:],Es66[2,:],labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{ANI} (kcal \times mol^{-1})$',plabel='CCSD(T)/CBS Trained NN',cmap=mpl.cm.viridis)
bin3 = aes.plot_corr_dist_axes(ax3,Es66[0,:],Es66[3,:],labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{ANI} (kcal \times mol^{-1})$',plabel='DFT/DZ Trained NN',cmap=mpl.cm.viridis)
bin4 = aes.plot_corr_dist_axes(ax4,Es66[0,:],Es66[4,:],labelx=r'$E_{CCSD(T)/CBS} (kcal \times mol^{-1})$',labely=r'$E_{DFT} (kcal \times mol^{-1})$',plabel='DFT/DZ QM',cmap=mpl.cm.viridis)

f[0].subplots_adjust(right=0.8)
cbar_ax = f[0].add_axes([0.825, 0.15, 0.025, 0.7])
cb1 = f[0].colorbar(bin1[-1], cmap=mpl.cm.viridis, cax=cbar_ax)

cb1.set_label('Count', fontsize=20)
plt.show()